# Advent of Code 2021

This notebook shows my solutions for Advent of Code 2021.

### Utility functions

In [1]:
import os

from typing import Any
from typing import Tuple

In [2]:
def parse_input(day: int) -> Tuple[str]:
    data_folder = "../data/aoc2021/"
    input_filename = os.path.join(data_folder, f"day_{day}_input.txt")
    with open(input_filename) as fp:
        return tuple([line.strip() for line in fp.readlines()])
    
def convert(data: Tuple[str], data_type=str) -> Tuple[Any]:
    return tuple(map(data_type, data))

### [Day 1: Sonar Sweep](https://adventofcode.com/2021/day/1)

**Part 1:** 
- Count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.)

In [3]:
in_1 = convert(parse_input(1), data_type=int)

in_1[:5]

(155, 157, 156, 172, 170)

In [4]:
def count_depth_increaments(data: Tuple[int]) -> int:
    depth_increaments = 0
    for i in range(1, len(data)):
        if data[i] > data[i - 1]:
            depth_increaments += 1
    return depth_increaments

In [5]:
print(f"{in_1[:5]}")

count_depth_increaments(in_1[:5])

(155, 157, 156, 172, 170)


2

In [6]:
print(f"{in_1[:10]}")

count_depth_increaments(in_1[:10])

(155, 157, 156, 172, 170, 186, 198, 189, 207, 213)


6

In [7]:
len(in_1)

2000

In [8]:
count_depth_increaments(in_1)

1713

**Part 2:** 
- Consider sums of a three-measurement sliding window.
- Count the number of times the sum of measurements in this sliding window increases from the previous sum.

In [9]:
def sliding_window_sums(
    data: Tuple[int],
    window_size: int = 3,
    debug: bool = False
) -> Tuple[int]:
    sliding_window = list()
    last_idx = len(data) - window_size + 1
    for i in range(last_idx):
        window_sum = sum(data[i:i + window_size])
        if debug:
            print(f"{i}-{i + window_size - 1}: {window_sum}")
        sliding_window.append(window_sum)
        
    return sliding_window

In [10]:
sliding_window_sums(in_1[:16], debug=True)

0-2: 468
1-3: 485
2-4: 498
3-5: 528
4-6: 554
5-7: 573
6-8: 594
7-9: 609
8-10: 642
9-11: 663
10-12: 679
11-13: 684
12-14: 676
13-15: 673


[468, 485, 498, 528, 554, 573, 594, 609, 642, 663, 679, 684, 676, 673]

In [11]:
sliding_window_depth = sliding_window_sums(in_1)

len(sliding_window_depth)

1998

In [12]:
count_depth_increaments(sliding_window_depth)

1734

### [Day 2: Dive!](https://adventofcode.com/2021/day/2)

**Part 1:**
- Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [13]:
in_2 = parse_input(2)

in_2[:10]

('forward 1',
 'down 5',
 'down 6',
 'down 2',
 'forward 8',
 'up 3',
 'up 2',
 'down 2',
 'forward 9',
 'forward 7')

In [14]:
def move_instruction(text: str) -> Tuple[str, float]:
    instruction, units = text.split(" ")
    return instruction, float(units)

In [15]:
movement_instructions = convert(in_2, data_type=move_instruction)

movement_instructions[:10]

(('forward', 1.0),
 ('down', 5.0),
 ('down', 6.0),
 ('down', 2.0),
 ('forward', 8.0),
 ('up', 3.0),
 ('up', 2.0),
 ('down', 2.0),
 ('forward', 9.0),
 ('forward', 7.0))

In [16]:
def calc_position(
    instructions: Tuple[Tuple[str, float]],
    debug: bool = False,
) -> int:
    horizontal_pos = depth = 0
    for op, value in instructions:
        if op == "forward":
            horizontal_pos += value
        elif op == "down":
            depth += value
        elif op == "up":
            depth -= value
        if debug:
            print(f"{op} {value}")
            print(f"Pos: {horizontal_pos}")
            print(f"Depth: {depth}\n")
    return int(horizontal_pos * depth)

In [17]:
calc_position(movement_instructions[:10], debug=True)

forward 1.0
Pos: 1.0
Depth: 0

down 5.0
Pos: 1.0
Depth: 5.0

down 6.0
Pos: 1.0
Depth: 11.0

down 2.0
Pos: 1.0
Depth: 13.0

forward 8.0
Pos: 9.0
Depth: 13.0

up 3.0
Pos: 9.0
Depth: 10.0

up 2.0
Pos: 9.0
Depth: 8.0

down 2.0
Pos: 9.0
Depth: 10.0

forward 9.0
Pos: 18.0
Depth: 10.0

forward 7.0
Pos: 25.0
Depth: 10.0



250

In [18]:
calc_position(movement_instructions)

1947824

**Part 2:**
- New interpretation:
  - down X increases your aim by X units.
  - up X decreases your aim by X units.
  - forward X does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.
- Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [19]:
def refined_calc_position(
    instructions: Tuple[Tuple[str, float]],
    debug: bool = False,
) -> int:
    horizontal_pos = depth = aim = 0
    for op, value in instructions:
        if op == "forward":
            horizontal_pos += value
            depth += aim * value
        elif op == "down":
            aim += value
        elif op == "up":
            aim -= value
        if debug:
            print(f"{op} {value}")
            print(f"Pos: {horizontal_pos}")
            print(f"Aim: {aim}")
            print(f"Depth: {depth}\n")
            
    return int(horizontal_pos * depth)

In [20]:
refined_calc_position(movement_instructions[:10], debug=True)

forward 1.0
Pos: 1.0
Aim: 0
Depth: 0.0

down 5.0
Pos: 1.0
Aim: 5.0
Depth: 0.0

down 6.0
Pos: 1.0
Aim: 11.0
Depth: 0.0

down 2.0
Pos: 1.0
Aim: 13.0
Depth: 0.0

forward 8.0
Pos: 9.0
Aim: 13.0
Depth: 104.0

up 3.0
Pos: 9.0
Aim: 10.0
Depth: 104.0

up 2.0
Pos: 9.0
Aim: 8.0
Depth: 104.0

down 2.0
Pos: 9.0
Aim: 10.0
Depth: 104.0

forward 9.0
Pos: 18.0
Aim: 10.0
Depth: 194.0

forward 7.0
Pos: 25.0
Aim: 10.0
Depth: 264.0



6600

In [21]:
refined_calc_position(movement_instructions)

1813062561